In [1]:
import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [2]:
lines=pd.read_csv("/workspaces/text-sql/experimentations/train.csv",encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/text-sql/train.csv'

In [ ]:
lines

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan =...
1,What is the current series where the new serie...,SELECT Current series FROM table WHERE Notes =...
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory...
3,Name the background colour for the Australian ...,SELECT Text/background colour FROM table WHERE...
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM tabl...
...,...,...
56350,What time was the match played with a score of...,SELECT Time FROM table WHERE Score = 3-2
56351,On which ground did the team play Aston Villa?,SELECT Ground FROM table WHERE Opponent = asto...
56352,What kind of competition was it at San Siro at...,SELECT Competition FROM table WHERE Ground = s...
56353,What is the total number of decile for the red...,SELECT COUNT Decile FROM table WHERE Name = re...


#### Checking null and dropping duplicates

In [ ]:
lines.isnull().sum()

question    0
sql         0
dtype: int64

In [ ]:
lines.drop_duplicates(inplace=True)

#### Converting to lower case

In [ ]:
lines['question']=lines['question'].apply(lambda x: x.lower())
lines['sql']=lines['sql'].apply(lambda x: x.lower())

#### Strip Extra Spaces

In [ ]:
lines['question']=lines['question'].apply(lambda x: x.strip())
lines['sql']=lines['sql'].apply(lambda x: x.strip())

#### Removing punctuations

In [ ]:
exclude = set(string.punctuation)
lines['question']=lines['question'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['sql']=lines['sql'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

#### Removing + signs

In [ ]:
lines['question']=lines['question'].apply(lambda x: re.sub(" +", " ", x))
lines['sql']=lines['sql'].apply(lambda x: re.sub(" +", " ", x))

#### Adding start and end to the queries

In [ ]:
lines['sql'] = lines['sql'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
all_eng_words=set()
for eng in lines['question']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [ ]:
all_sql_words=set()
for i in lines['sql']:
    for word in i.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

In [ ]:
lines['length_question_sentence']=lines['question'].apply(lambda x:len(x.split(" ")))
lines['length_sql_sentence']=lines['sql'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()

,question,sql,length_question_sentence,length_sql_sentence
0,tell me what the notes are for south australia,START_ select notes from table where current s...,9,11
1,what is the current series where the new serie...,START_ select current series from table where ...,13,15
2,what is the format for south australia,START_ select format from table where stateter...,7,10
3,name the background colour for the australian ...,START_ select textbackground colour from table...,9,12
4,how many times is the fuel propulsion is cng,START_ select count fleet series quantity from...,9,13


In [ ]:
max_length_src=max(lines['length_sql_sentence'])
max_length_tar=max(lines['length_question_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_sql_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_sql_words)
num_encoder_tokens, num_decoder_tokens

(37411, 35441)

In [ ]:
num_decoder_tokens += 1

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = lines.sample(frac=1)

lines.head()

,question,sql,length_question_sentence,length_sql_sentence
0,what is the race when the jockey is frankie de...,START_ select race from table where jockey fra...,21,16
1,if the date is 7212006 who was the opponent,START_ select opponent from table where date 7...,9,9
2,how many marks did he get in 2005,START_ select marks from table where season 20...,8,9
3,when hamilton academical is the opponent what ...,START_ select count attendance from table wher...,11,11
4,who was the manufacturer for the car that dani...,START_ select manufacturer from table where la...,16,13


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
df= lines[:10000]